In [1]:
import os
import ee
import geemap
import time
from geemap.datasets import DATA, get_metadata

In [2]:
#geemap.update_package()

In [3]:
ee.Initialize()

In [4]:
#MERRA2 AOD
DATA.NASA_GSFC_MERRA_aer_2

'NASA/GSFC/MERRA/aer/2'

In [5]:
get_metadata(DATA.NASA_GSFC_MERRA_aer_2)

HTML(value='\n            <html>\n            <body>\n                <h3>MERRA-2 M2T1NXAER: Aerosol Diagnosti…

In [ ]:
# AP Bounding Box
roi = ee.Geometry.Rectangle([34.5, 12.1111, 59.8, 32.0])

In [ ]:
# Set output directory
base_out_dir = r"Z:\Thesis\Data\GEE\MERRA2_aer\AOD_ducmass"
if not os.path.exists(base_out_dir):
    os.makedirs(base_out_dir)

In [ ]:
# Function to download data for a specific year with retry logic
def download_for_year(year, retries=100):
    start_date = f'{year}-01-01'
    end_date = f'{year+1}-01-01'
    
    # Filter image collection
    collection = ee.ImageCollection(DATA.NASA_GSFC_MERRA_aer_2) \
        .filterDate(start_date, end_date) \
        .filterBounds(roi)
    
    # Output directory for the year
    out_dir = os.path.join(base_out_dir, str(year))
    
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    
    # Retry logic
    attempt = 0
    while attempt < retries:
        try:
            # Export image collection for the current year
            geemap.ee_export_image_collection(collection, out_dir=out_dir, region=roi)
            print(f"Download for {year} successful.")
            break  # Exit the loop if successful
        except Exception as e:
            attempt += 1
            print(f"Error downloading data for {year}: {e}. Retrying {attempt}/{retries}...")
            time.sleep(5)  # Wait for 5 seconds before retrying
    else:
        print(f"Failed to download data for {year} after {retries} attempts.")

In [ ]:
# Loop through the years 1985-2015
for year in range(1980, 2024):
    download_for_year(year)

In [ ]:
###############################################################################################################################

In [ ]:
#NASA_GSFC_MERRA_aer_2
collection = ee.ImageCollection(DATA.NASA_GSFC_MERRA_aer_2) \
    .filterDate('1985-01-01', '1986-01-01') \
    .filterBounds(roi) 

In [ ]:
image = collection.first()

In [ ]:
Map = geemap.Map()
Map.centerObject(roi)
Map

In [ ]:
#Seperate band Visualization
duc_mass_vis = {
    'min': image.reduceRegion(
        reducer=ee.Reducer.min(), 
        geometry=roi, 
        scale=1000, 
        maxPixels=1e9  # Increase maxPixels
    ).get('DUCMASS').getInfo(), 
    'max': image.reduceRegion(
        reducer=ee.Reducer.max(), 
        geometry=roi, 
        scale=1000, 
        maxPixels=1e9  # Increase maxPixels
    ).get('DUCMASS').getInfo(),
    'palette': ['001137', '01abab', 'e7eb05', '620500']
}

Map.addLayer(image.select('DUCMASS'), duc_mass_vis, 'DUCMASS', opacity=0.6)

In [ ]:
# Add Legend

# Define the legend dictionary
legend_dict = {
    'Very Low': '001137',
    'Low': '01abab',
    'High': 'e7eb05',
    'Very High': '620500'
}

# Add the legend to the map
Map.add_legend(legend_title='DUCMASS Legend', legend_dict=legend_dict, position='bottomright')

In [ ]:
# Set output directory
out_dir = r"Z:\Thesis\Data\GEE\MERRA2_aer\AOD_ducmass\1985"

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [ ]:
#loc = ee.Geometry.Rectangle([34.5, 12.1111, 59.8, 32.0])
collection = ee.ImageCollection(DATA.NASA_GSFC_MERRA_aer_2) \
    .filterBounds(roi) \
    .filterDate('1985-01-01', '1986-01-01') \
    #.filter(ee.Filter.listContains())

In [ ]:
collection.aggregate_array('system:index').getInfo()

In [ ]:
geemap.ee_export_image_collection(collection, out_dir=out_dir, region=roi)